## Import irrevelnt libraries

In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

## Data

In [2]:
mnist_dataset, mnist_info = tfds.load(name = 'mnist', with_info = True, as_supervised = True)
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']
num_validation_samples = 0.1*mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)
num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

In [3]:
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255
    return image, label

In [4]:
scaled_train_and_validation_data = mnist_train.map(scale)
test_data = mnist_test.map(scale)

In [5]:
BUFFER_SIZE = 10000
shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)
validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

In [6]:
BATCH_SIZE = 100
train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)
validation_inputs, validation_targets = next(iter(validation_data))

## Model

### Outline the model

In [11]:
input_size = 784
output_size = 10
hidden_layer_size = 50

In [12]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape = (28, 28, 1)), # input_layer
    tf.keras.layers.Dense(hidden_layer_size, activation = 'relu'), # 1st hidden layer
    tf.keras.layers.Dense(hidden_layer_size, activation = 'tanh'), # 2nd hidden layer
    tf.keras.layers.Dense(output_size, activation = 'softmax'), # output layer
])

### Choose the optimizer and the loss function

In [13]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


### Training

In [14]:
NUM_EPOCHS = 5
model.fit(train_data, epochs = NUM_EPOCHS, validation_data = (validation_inputs, validation_targets), verbose = 2)

Epoch 1/5
540/540 - 2s - loss: 0.3938 - accuracy: 0.8926 - val_loss: 0.1928 - val_accuracy: 0.9447
Epoch 2/5
540/540 - 1s - loss: 0.1628 - accuracy: 0.9509 - val_loss: 0.1372 - val_accuracy: 0.9603
Epoch 3/5
540/540 - 1s - loss: 0.1205 - accuracy: 0.9642 - val_loss: 0.1123 - val_accuracy: 0.9685
Epoch 4/5
540/540 - 1s - loss: 0.0956 - accuracy: 0.9715 - val_loss: 0.0947 - val_accuracy: 0.9708
Epoch 5/5
540/540 - 1s - loss: 0.0799 - accuracy: 0.9756 - val_loss: 0.0848 - val_accuracy: 0.9752


In [15]:
test_loss, test_accuracy = model.evaluate(test_data)

1/1 [==============================] - 0s 2ms/step - loss: 0.0993 - accuracy: 0.9697


In [16]:
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

Test loss: 0.10. Test accuracy: 96.97%
